In [ ]:
from nbtemplate import display_header, get_path, display_codetoggle
display_header('Place the detectors.ipynb')

In [ ]:
from __future__ import print_function, division
import sys
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.table import Table, join
import astropy.units as u
import marxs
from marxs import visualization

from marxs.source import PointSource, FixedPointing, JitterPointing
from marxs.analysis import resolvingpower_from_photonlist
from marxs.simulator import Sequence

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from marxslynx import lynx

In [ ]:
wave = np.arange(12., 50., .1) * u.Angstrom
energy = wave.to(u.keV, equivalencies = u.spectral())
flux = np.ones_like(wave)[1:] / np.abs(np.diff(energy))

mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                           energy={"energy": energy[::-1][1:], "flux": flux[::-1]},
                           flux=1.)
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(1e5)
photons = fixedpointing(photons)



In [ ]:
pl = lynx.PerfectLynx()

photons = pl(photons)
photons = photons[np.isfinite(photons['order'])]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ind = (photons['detcirc_phi'] > -0.1) & (photons['detcirc_phi'] < 0.3)
out = ax.hist(photons['detcirc_phi'][ind], weights=photons['probability'][ind], bins=50)

In [ ]:
ind.sum()

In [ ]:
import astropy.units as u
photons['wave'] = photons['energy'].to(u.Angstrom, equivalencies=u.spectral())
wavelim = np.arange(12., 49, 12)
px = [photons['proj_x'][(photons['wave'] > wavelim[i]) & (photons['wave'] < wavelim[i+1])] for i in range(len(wavelim)-1)]
weights = [photons['probability'][(photons['wave'] > wavelim[i]) & (photons['wave'] < wavelim[i+1])] for i in range(len(wavelim)- 1)]
labels = ['{}-{} nm'.format(wavelim[i]/10, wavelim[i+1]/10) for i in range(len(wavelim) -1)]

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
out = ax.hist(px, weights=weights, bins=np.arange(-400, 800, 30), histtype='barstacked', label=labels)
ax.legend()
ax.set_ylabel('Number of photons')
ax.set_xlabel('Distance from focal point [mm]')
fig.savefig(get_path('figures')+ '/detectorplacement.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/detectorplacement.pdf', bbox_inches='tight')


In [ ]:
#CCDs are about 49 mm long

In [ ]:
from marxs import optics
from marxslynx.lynx import conf


In [ ]:
from marxslynx.lynx import RowlandDetArray
conf['det_kwargs']['theta'] = [3.1, 3.18]
r = RowlandDetArray(conf)
len(r.elements), r.elements[0].geometry['center'], r.elements[-1].geometry['center']


In [ ]:
mysource = PointSource(coords=SkyCoord(0., 0., unit='deg'),
                           energy={"energy": np.arange(1., 10, 1), "flux": np.ones(9)},
                           flux=1.)
fixedpointing = FixedPointing(coords=SkyCoord(0., 0., unit='deg'))
photons = mysource.generate_photons(1e5)
photons = fixedpointing(photons)

In [ ]:
photons = pl.elements[0](photons)

In [ ]:
p = pl.elements[2](photons.copy())
p = pl.elements[3](p)
p = pl.elements[4](p)

In [ ]:
before = np.histogram(photons['energy'], weights=photons['probability'], bins=np.arange(1., 10, 1))
ind = p['order'] == 0
after = np.histogram(p['energy'][ind], weights=p['probability'][ind], bins=np.arange(1., 10, 1))

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)
frac = after[0] / before[0]
tsubaperangle = np.linspace(0, np.pi/2, 7)[1:]
for i, ang in enumerate(tsubaperangle):
    ax.plot(np.arange(1.5, 9.), 
             ang/np.pi * frac + (np.pi -ang)/np.pi, 
             label='{:3.0f}'.format(np.rad2deg(ang)))

ax.legend(title='Subaperture\nangle [deg]', loc='lower right')
ax.set_xlabel('energy [keV]')
ax.set_ylabel('photon fraction in 0th order')
fig.savefig(get_path('figures') + 'highen.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + 'highen.pdf', bbox_inches='tight')

In [ ]:
row = lynx.conf['rowland']
row.R, row.r, np.rad2deg(lynx.conf['alpha']), np.rad2deg(lynx.conf['beta'])

In [ ]:
display_codetogle()